<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2022-actors/blob/Policzenie-pierwszych-similarity/colabs/Wczytanie_danych.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup sparka

##Setup sparka

In [1]:
!pip install pyspark py4j
!pip install -q findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget ftp://ftp.ps.pl/pub/apache/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
!tar xf spark-3.3.1-bin-hadoop2.tgz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 44 kB/s 
     |████████████████████████████████| 200 kB 51.3 MB/s 
     |████████████████████████████████| 199 kB 48.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845514 sha256=7f0229ac31b7b7dad299a19b98412cb1052c92f6d77dba3d4abb3cd89f3900ec
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark
--2022-11-21 15:33:49--  ftp://ftp.ps.pl/pub/apache/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
           => ‘spark-3.3.1-bin-hadoop2.tgz’
Resolving ftp.ps.pl (ftp.ps.pl)... 212.14.28.36, 2001:4c58:62:2::36
Connecting to ftp.ps.pl (ftp.ps.pl)|212.14.28.36|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/apache/spark/spark-3.3.1 ... done.
==>

In [2]:
import pyspark
import findspark
from pyspark.sql import SparkSession
import os

In [3]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop2"
spark=SparkSession.builder.appName('Colab').getOrCreate()
spark

#Pobranie danych

##Import danych

In [4]:
!wget https://datasets.imdbws.com/name.basics.tsv.gz
!wget https://datasets.imdbws.com/title.akas.tsv.gz
!wget https://datasets.imdbws.com/title.basics.tsv.gz
!wget https://datasets.imdbws.com/title.crew.tsv.gz
!wget https://datasets.imdbws.com/title.episode.tsv.gz
!wget https://datasets.imdbws.com/title.principals.tsv.gz
!wget https://datasets.imdbws.com/title.ratings.tsv.gz

--2022-11-21 15:34:37--  https://datasets.imdbws.com/name.basics.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 13.226.210.101, 13.226.210.117, 13.226.210.114, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|13.226.210.101|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 236670295 (226M) [binary/octet-stream]
Saving to: ‘name.basics.tsv.gz’

name.basics.tsv.gz  100%[===================>] 225.71M  26.8MB/s    in 7.8s    

2022-11-21 15:34:45 (29.1 MB/s) - ‘name.basics.tsv.gz’ saved [236670295/236670295]

--2022-11-21 15:34:45--  https://datasets.imdbws.com/title.akas.tsv.gz
Resolving datasets.imdbws.com (datasets.imdbws.com)... 13.226.210.101, 13.226.210.117, 13.226.210.114, ...
Connecting to datasets.imdbws.com (datasets.imdbws.com)|13.226.210.101|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 287964961 (275M) [binary/octet-stream]
Saving to: ‘title.akas.tsv.gz’

title.akas.tsv.gz   100%[===================>

##Rozpakowanie danych

In [5]:
!gzip -dc /content/name.basics.tsv.gz > name.basics.csv
!gzip -dc /content/title.akas.tsv.gz > title.akas.csv
!gzip -dc /content/title.basics.tsv.gz > title.basic.csv
!gzip -dc /content/title.crew.tsv.gz > title.crew.csv
!gzip -dc /content/title.episode.tsv.gz > title.episode.csv
!gzip -dc /content/title.principals.tsv.gz > title.principals.csv
!gzip -dc /content/title.ratings.tsv.gz > title.ratings.csv

#Wczytanie danych

##Wczytajmy dane z rozpakowanych plików

In [6]:
df_name_basics=spark.read.option("header", "true").option("delimiter", "\t").csv('name.basics.csv' ) 
df_title_akas=spark.read.option("header","true").option("delimiter", "\t").csv('title.akas.csv')
df_title_basic=spark.read.option("header","true").option("delimiter", "\t").csv('title.basic.csv')
df_title_crew=spark.read.option("header","true").option("delimiter", "\t").csv('title.crew.csv')
df_title_episode=spark.read.option("header","true").option("delimiter","\t").csv('title.episode.csv')
df_title_principals=spark.read.option("header","true").option("delimiter","\t").csv('title.principals.csv')
df_title_ratings=spark.read.option("header","true").option("delimiter","\t").csv('title.ratings.csv')

##Wyświetlmy dane

In [7]:
df_name_basics.show(3)
df_title_akas.show(3)
df_title_basic.show(3)
df_title_crew.show(3)
df_title_episode.show(3)
df_title_principals.show(3)
df_title_ratings.show(3)

+---------+---------------+---------+---------+--------------------+--------------------+
|   nconst|    primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+---------+---------------+---------+---------+--------------------+--------------------+
|nm0000001|   Fred Astaire|     1899|     1987|soundtrack,actor,...|tt0072308,tt00531...|
|nm0000002|  Lauren Bacall|     1924|     2014|  actress,soundtrack|tt0071877,tt00373...|
|nm0000003|Brigitte Bardot|     1934|       \N|actress,soundtrac...|tt0056404,tt00573...|
+---------+---------------+---------+---------+--------------------+--------------------+
only showing top 3 rows

+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
|  titleId|ordering|               title|region|language|      types|   attributes|isOriginalTitle|
+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
|tt0000001|       1|          Карменсіта|    

# Złączenie istotnych kolumn

## Wybierzmy następujące kolumny z tabel:
- df_title_basic:
  - tconst (unikalny numer tytułu - potrzebne do złączenia tabel)
  - titleType (rodzaj tytułu, np. film, serial - można porównywać podobieństwo aktorów na podstawie tego w jakich typach produkcji grali)
  - originalTitle (tytuł produkcji - być może przyda się do analizy podobnych filmów)
  - isAdult (czy produkcja jest dla dorosłych - można wykorzystać do porównywania aktorów na podstawie tego czy grają głównie w produkcjach dla dorosłych)
  - genres (gatunki - może posłużyć do porównania aktorów na podstawie tego, że grali w produkcjach o podobnych gatunkach)

- df_title_principals:
  - tconst (unikalny numer tytułu - potrzebne do złączenia tabel)
  - nconst (unikalny numer aktora - potrzebne do złączenia tabel)
  - category (kategoria pracy (actor/actress) - może posłużyć do porównania aktorów na podstawie takiej samej płci)
  - characters (postacie zagrane w produkcji - może posłużyć do porównania aktorów na podstawie podobnych granych postaci)

- df_name_basics:
  - nconst(unikalny numer aktora - potrzebne do złączenia tabel)
  - primaryName (imię i nazwisko aktora - potrzebne do wyświetlania aktora lub do odnajdywania go w bazie po imieniu i nazwisku)
  - birthYear (rok urodzenia aktora - może posłużyć do porównania aktorów na podstawie podobnych okresów, w których żyli i grali)
  - deathYear ( rok śmierci aktora - może posłużyć do porównania aktorów na podstawie podobnych okresów, w których żyli i grali)
  - knownForTitles (tytuły produkcji, z których znany jets aktor - może posłużyć do porównania aktorów na podstawie tych samych lub podobnych produkcji, z których są znani)

In [8]:
df_title_basic_selected = df_title_basic.select(["tconst", "titleType", "originalTitle", "isAdult", "genres"])
df_title_principals_selected = df_title_principals.select(["tconst", "nconst", "category", "characters"])
df_name_basics_selected = df_name_basics.select(["nconst", "primaryName", "birthYear", "deathYear", "knownForTitles"])

## Wybierzmy z tabeli df_title_principals_selected tylko aktorów i aktorki

In [9]:
df_title_principals_selected = df_title_principals_selected.filter((df_title_principals_selected.category == "actor") | (df_title_principals_selected.category == "actress"))

# Złączmy tabele df_name_basics, df_title_principals i df_title_basic

In [10]:
print("df_name_basics_selected dataframe size: ", (df_name_basics_selected.count(), len(df_name_basics_selected.columns)))
print("df_title_principals_selected dataframe size: ", (df_title_principals_selected.count(), len(df_title_principals_selected.columns)))
print("df_title_basic_selected dataframe size: ", (df_title_basic_selected.count(), len(df_title_basic_selected.columns)))
data = df_name_basics_selected.join(df_title_principals_selected, "nconst", "inner")
print("joined dataframe size: ", (data.count(), len(data.columns)))
data = data.join(df_title_basic_selected, "tconst", "left")
print("joined dataframe size: ", (data.count(), len(data.columns)))

df_name_basics_selected dataframe size:  (12091560, 5)
df_title_principals_selected dataframe size:  (20807550, 4)
df_title_basic_selected dataframe size:  (9386465, 5)
joined dataframe size:  (20803919, 8)
joined dataframe size:  (20803919, 12)


# Wybierzmy przykładowe wiersze z końcowego dataframe'u

In [11]:
sample = data.rdd.takeSample(False, 5)

In [12]:
import pandas as pd
sample = pd.DataFrame(sample, columns = data.columns)
sample

,tconst,nconst,primaryName,birthYear,deathYear,knownForTitles,category,characters,titleType,originalTitle,isAdult,genres
0,tt16653766,nm12193060,Mirna Mihelcic,\N,\N,"tt11944922,tt13749650,tt16526150,tt16636212",actress,"[""Marusa""]",tvEpisode,Mrkomir i blagdani,0,"Comedy,History"
1,tt16391860,nm13205161,Roberto Ambrosini,\N,\N,tt16391860,actor,"[""Ambrosini, Roberto""]",short,Montagne di plastica,0,"Documentary,Short"
2,tt1801021,nm0033312,Taís Araújo,1978,\N,"tt0809665,tt2372755,tt0407377,tt0138277",actress,"[""Helena Toledo""]",tvEpisode,Episode #1.116,0,"Comedy,Drama,Romance"
3,tt15418100,nm0032122,Cayito Aponte,1938,2018,"tt0092981,tt0084615,tt1772630,tt4603738",actor,\N,tvEpisode,Episode #1.1651,0,Comedy
4,tt0648757,nm2086729,Isa Gonçalves,\N,\N,"tt1435193,tt0411665,tt0407418,tt1542802",actress,"[""Rita""]",tvEpisode,Neco Quer Ser Actor,0,Adventure


# Similarity

## Deifniujemy metodę liczącą podobieństwo dwóch aktorów

In [54]:
def similarity(actor_1_id, actor_2_id):
  # tworzymy słownik i zmienną similarity
  actors_dict = [{'actor_id': actor_1_id}, {'actor_id': actor_2_id}]
  similarity = 0

  # załadowanie danych o aktorach do odpowiednich tabel
  data_1_2 = data.filter((data.nconst == actor_1_id) | (data.nconst == actor_2_id))
  actor_1_data = data.filter(data_1_2.nconst == actor_1_id)
  actor_2_data = data.filter(data_1_2.nconst == actor_2_id)
  actor_firsts = [actor_1_data.first(), actor_2_data.first()]
  
  # załadowanie danych do odpowiednich słowników
  for i in [0, 1]:    
    actors_dict[i]['knownForTiles'] = actor_firsts[i].knownForTitles.split(",")
    # gdyby brakowało informacji o dacie urodzenia
    try:
      bYear = int(actor_firsts[i].birthYear)
      dYear = int(actor_firsts[i].deathYear)
      # TO DO usunąć funkcję int(), gdy typ danych w kolumnach birthYear i deathYear zostanie zmieniony na poprawny
      # trzeba będzie wtedy też zmienić metodę radzenia sobie z brakującymi wartościami
    except ValueError:
      bYear = 0
      dYear = 0
    actors_dict[i]['yearsActivity'] = [bYear, dYear]
    
  actors_dict[0]['titles_id'] = actor_1_data.select('tconst').rdd.flatMap(lambda x: x).collect()
  actors_dict[1]['titles_id'] = actor_2_data.select('tconst').rdd.flatMap(lambda x: x).collect()
  
  # similairty na podstawie występowania w tych samych tytułach
  for title in actors_dict[0]['titles_id']:
    if title in actors_dict[1]['titles_id']:
      similarity += 0.25
  
  # similairty na podstawie bycia znanym z tych samych tytułów
  for title in actors_dict[0]['knownForTiles']:
    if title in actors_dict[1]['knownForTiles']:
      similarity += 3

  # similairty na podstawie życia w tym samym okresie czasu
  years_overlap = max(0, min(actors_dict[0]['yearsActivity'][1], actors_dict[1]['yearsActivity'][1])- max(actors_dict[0]['yearsActivity'][0], actors_dict[1]['yearsActivity'][0]))
  similarity += years_overlap * 0.02
  # TO DO zoptymalizować przechodzenie przez listę lat
  
  return similarity
  # TO DO zestandaryzować wartośc zwracaną przez metodę do wartości z przedziału [-1, 1]

## Liczymy pierwsze similarity dla trzech par aktorów

In [55]:
aktor1 = "nm0000010"
aktor2 = "nm0000011"
print(f"Podobieństwo dla aktorów {aktor1} i {aktor2} wynosi {round(similarity(aktor1, aktor2), 3)}")

Podobieństwo dla aktorów nm0000010 i nm0000011 wynosi 1.2


In [56]:
aktor3 = "nm0000012"
aktor4 = "nm0000034"
print(f"Podobieństwo dla aktorów {aktor3} i {aktor4} wynosi {round(similarity(aktor3, aktor4), 3)}")

Podobieństwo dla aktorów nm0000012 i nm0000034 wynosi 1.26


In [57]:
aktor5 = "nm12193060"
aktor6 = "nm0032122"
print(f"Podobieństwo dla aktorów {aktor5} i {aktor6} wynosi {round(similarity(aktor5, aktor6), 3)}")

Podobieństwo dla aktorów nm12193060 i nm0032122 wynosi 0.0
